In [1]:
import os
os.environ['KERAS_BACKEND'] = 'jax'

# Built-in imports:
from typing import List
from itertools import islice

# Dependency imports: 
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot

# Import the GravyFlow module:
import gravyflow as gf

## Obtaining Transient Events

To acquire data from specific gravitational wave events (transients), use `gf.TransientObtainer`. This works similarly to `gf.NoiseObtainer` but is specifically designed for acquiring data around known event times.

### TransientObtainer

**Parameters:**

- `ifo_data_obtainer` : `gf.IFODataObtainer` (**required**):
  > The IFODataObtainer configured for transient acquisition. Unlike `NoiseObtainer`, this parameter is mandatory. The `data_labels` should include `gf.DataLabel.EVENTS` or `gf.DataLabel.GLITCHES` (not `gf.DataLabel.NOISE`).

- `ifos` : Union[`gf.IFO`, List[`gf.IFO`]] = `[gf.IFO.L1]`:
  > List of interferometers to acquire data from.

- `event_names` : Union[str, List[str]] = None:
  > Optional event name(s) to fetch (e.g., `"GW150914"` or `["GW150914", "GW170817"]`). If set, only data for these specific events will be returned, superseding the default behavior of returning all events. Event names must match those in GWTC catalogs.

- `event_types` : `List[gf.EventType]` = `[gf.EventType.CONFIDENT]`:
  > Filter by event confidence.
  > **Options:**
  > - `gf.EventType.CONFIDENT`: Confirmed detections (Default).
  > - `gf.EventType.MARGINAL`: Marginal triggers/candidates.

- `data_labels` : List[`gf.DataLabel`] = `[gf.DataLabel.EVENTS]`:
  > Specifies which transient types to include. Must NOT include `gf.DataLabel.NOISE` (raises `ValueError`). For noise acquisition, use `gf.NoiseObtainer` instead.

- `groups` : dict = `{"all": 1.0}`:
  > Group splits for data partitioning. Defaults to a single "all" group (no train/val/test split), which is typical for transient evaluation.

In [2]:
transient_obtainer = gf.TransientObtainer(
    ifo_data_obtainer=gf.IFODataObtainer(
        observing_runs=gf.ObservingRun.O3,
        data_quality=gf.DataQuality.BEST,
        data_labels=[gf.DataLabel.EVENTS]
    ),
    ifos=[gf.IFO.H1, gf.IFO.L1],
    event_names=["GW150914", "GW170817"]  # Optional: specific events
)

## Searching for Events

GravyFlow provides a powerful `search_events` function to filter gravitational wave events from GWTC catalogs based on astrophysical properties, observing runs, and more.

**Parameters:**

- `source_type` : `Union[gf.SourceType, str]` = `None`:
  > Filter by astrophysical source type. 
  > **Enums (Recommended):**
  > - `gf.SourceType.BBH`: Binary Black Hole (both masses ≥ 3 M☉)
  > - `gf.SourceType.BNS`: Binary Neutron Star (both masses < 3 M☉)
  > - `gf.SourceType.NSBH`: Neutron Star - Black Hole (one < 3 M☉, one ≥ 3 M☉)
  >
  > **Strings (Supported):** `"BBH"`, `"BNS"`, `"NSBH"` (case-insensitive).

- `observing_runs` : `List[gf.ObservingRun]` = `None`:
  > Filter by specific observing runs (e.g., `[gf.ObservingRun.O3]`).

- `mass1_range` : `tuple` = `None`:
  > (min, max) range for primary mass in solar masses. Use `None` for unbounded limits. 
  > *Example:* `(30, None)` finds events with m1 > 30 M☉.

- `mass2_range` : `tuple` = `None`:
  > (min, max) range for secondary mass in solar masses.

- `total_mass_range` : `tuple` = `None`:
  > (min, max) range for total system mass (m1 + m2).

- `distance_range` : `tuple` = `None`:
  > (min, max) range for luminosity distance in Mpc.
  > *Example:* `(None, 500)` finds events closer than 500 Mpc.

- `name_contains` : `str` = `None`:
  > Substring to search for in the event name (case-insensitive).
  > *Example:* `"GW17"` matches all 2017 events.

**Returns:**
- `List[str]`: A list of event names matching all specified conditions.

In [3]:
### Examples

#### 1. Filter by Source Type (Using Enums)

# Find all Binary Neutron Star events
bns_events = gf.search_events(source_type=gf.SourceType.BNS)
print(bns_events)
# Output: ['GW170817', 'GW190425']

#### 2. Filter by Observing Run
# Find all Binary Black Holes in O3
o3_bbh = gf.search_events(
    source_type=gf.SourceType.BBH,
    observing_runs=[gf.ObservingRun.O3]
)

#### 3. Complex Physical Queries
# Find heavy BBHs (Total Mass > 80 M☉) that are relatively close (< 1000 Mpc)
heavy_nearby = gf.search_events(
    source_type=gf.SourceType.BBH,
    total_mass_range=(80, None),
    distance_range=(None, 1000)
)

#### 4. Search by Name
# Find all events from 2017
events_2017 = gf.search_events(name_contains="GW17")


2025-12-11 18:21:45,126 - INFO - Fetched 197 events with PE parameters
2025-12-11 18:21:45,130 - INFO - Fetched 100 events with PE parameters
2025-12-11 18:21:45,133 - INFO - Fetched 197 events with PE parameters
2025-12-11 18:21:45,136 - INFO - Fetched 197 events with PE parameters


['GW170817', 'GW190425']


In [4]:
print(len(gf.search_events(observing_runs=[gf.ObservingRun.O1])))
print(len(gf.search_events(observing_runs=[gf.ObservingRun.O2])))
print(len(gf.search_events(observing_runs=[gf.ObservingRun.O3])))
print(len(gf.search_events(observing_runs=[gf.ObservingRun.O4])))

2025-12-11 18:21:45,146 - INFO - Fetched 3 events with PE parameters
2025-12-11 18:21:45,149 - INFO - Fetched 8 events with PE parameters
2025-12-11 18:21:45,152 - INFO - Fetched 100 events with PE parameters
2025-12-11 18:21:45,155 - INFO - Fetched 86 events with PE parameters


3
8
100
86


In [ ]:
onsource, offsource, gps_times = next(gf.TransientObtainer(
    ifo_data_obtainer=gf.IFODataObtainer(
        observing_runs=gf.ObservingRun.O1,
        data_quality=gf.DataQuality.BEST,
        data_labels=[gf.DataLabel.EVENTS]
    ),
    ifos=[gf.IFO.H1, gf.IFO.L1],
    event_names=["GW150914", "GW170817"]  # Optional: specific events
)())


print(type(offsource), type(onsource), type(gps_times))
whitened_onsource = gf.whiten(onsource, offsource, 2048.0,                 fft_duration_seconds=1.0,
                overlap_duration_seconds=0.5,
                filter_duration_seconds=1.0)

onsource *= 1E21
offsource *= 1E21

print(type(whitened_onsource))

2025-12-11 18:32:16,427 - INFO - Fetched 197 events with PE parameters
2025-12-11 18:32:16,428 - INFO - 
FEATURE MODE: Auto-precaching 2 feature segments
Estimated time: ~0.1 minutes (4 seconds)
Padding: 32.0s each side


<class 'jaxlib._jax.ArrayImpl'> <class 'jaxlib._jax.ArrayImpl'> <class 'jaxlib._jax.ArrayImpl'>
<class 'jaxlib._jax.ArrayImpl'>


In [10]:
gw150914_plot_uw = gf.generate_strain_plot(
    {"Onsource Noise": offsource[0]},
    title=[
        f"Onsource Real L1 Noise at {gps_times[0][0]:.0f} s",
    ]
)



gw150914_plot = gf.generate_strain_plot(
    {"Onsource Noise": whitened_onsource[0]},
    title=[
        f"Onsource Real L1 Noise at {gps_times[0][0]:.0f} s",
    ]
)

output_notebook()
show(gw150914_plot)
show(gw150914_plot_uw)

Loading BokehJS ...